# Digit Recognizer Training
Trainig part of [Kaggle Digit Reconizer](https://www.kaggle.com/c/digit-recognizer/overview).

1. [Compute SVD with Spark Row Matrix](https://spark.apache.org/docs/2.2.0/mllib-dimensionality-reduction.html#singular-value-decomposition-svd) to decide the number of principal components (k) to use for PCA.
2. [PCA on Spark](https://spark.apache.org/docs/2.2.0/mllib-dimensionality-reduction.html#principal-component-analysis-pca) to reduce dimensions to k.
3. [Random Forest Classifer on Spark](https://spark.apache.org/docs/latest/ml-classification-regression.html#random-forest-classifier) to train a model. 

# Setup

In [1]:
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Row

import org.apache.spark.ml.feature._
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification._

import org.apache.spark.ml.linalg._
import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.linalg.distributed._


import java.time.{Period, LocalDate, Instant}
import java.sql.Timestamp
import java.time.temporal.ChronoUnit

In [2]:
import scala.collection.mutable.ListBuffer

val timing = new StringBuffer
val times = new ListBuffer[Long]()

def clear(): Unit = {
    timing.setLength(0)
    times.clear
}
def average(): Long = {
    times.reduce(_+_) / times.length
}

/**
@param label Description about the run
@code code to execute
@return execution
*/
def timed[T](label: String, code: => T): T = {
    val start = System.currentTimeMillis()
    val result = code
    val stop = System.currentTimeMillis()
    timing.append(s"Processing $label took ${stop - start} ms.\n")
    times.append(stop - start)
    result
}

timing = 
times = ListBuffer()


clear: ()Unit
average: ()Long
timed: [T](label: String, code: => T)T


ListBuffer()

In [3]:
%%html
<!-- To left align the HTML components in Markdown -->
<style>
table {float:left}
</style>

<console>:60: error: missing argument list for method timed
Unapplied methods are only converted to functions when a function type is expected.
You can make this conversion explicit by writing `timed _` or `timed(_,_)` instead of `timed`.
       timed
       ^
lastException: Throwable = null


<!-- To left align the HTML components in Markdown -->

In [4]:
val NUM_CORES = 4
val NUM_PARTITIONS = 4

spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
/*
spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.driver.memory", "6g")
spark.conf.set("spark.executor.memory", "2g")
spark.conf.set("spark.master", "spark://masa:7077")
*/

Waiting for a Spark session to start...

NUM_CORES = 8
NUM_PARTITIONS = 4


4

In [6]:
val configMap = spark.conf.getAll.foreach(println)

(spark.serializer,org.apache.spark.serializer.KryoSerializer)
(spark.driver.host,10.115.130.67)
(spark.eventLog.enabled,true)
(spark.driver.port,32793)
(spark.hadoop.validateOutputSpecs,True)
(spark.repl.class.uri,spark://10.115.130.67:32793/classes)
(spark.jars,file:/home/oonisim/.local/share/jupyter/kernels/apache_toree_scala/lib/toree-assembly-0.3.0-incubating.jar)
(spark.repl.class.outputDir,/tmp/spark-854365bf-9f7d-4a6b-bbb4-76af2232bcb3/repl-335596a3-b186-46d8-ab2f-66c1945f4ad8)
(spark.app.name,Apache Toree)
(spark.driver.memory,1g)
(spark.executor.instances,3)
(spark.history.fs.logdirectory,hdfs://oonisim:8020/logs_spark)
(spark.default.parallelism,32)
(spark.executor.id,driver)
(spark.submit.deployMode,client)
(spark.master,yarn)
(spark.ui.filters,org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter)
(spark.executor.memory,4g)
(spark.eventLog.dir,hdfs://oonisim:8020/logs_spark)
(spark.executor.cores,4)
(spark.driver.appUIAddress,http://10.115.130.67:4040)
(spark.org.apach

configMap: Unit = ()


## Constant

In [7]:
val PROTOCOL="hdfs://"
val DATA_DIR="/data/kaggle/mnist/data"
val MODEL_DIR="/data/kaggle/mnist/model"
val RESULT_DIR="/data/kaggle/mnist/result"

val CROSS_VALIDATE_NUM_HOLDS = 5
val CROSS_VALIDATE_PARALLELISM = NUM_CORES

val COVERAGE:Double = 0.95

PROTOCOL = hdfs://
DATA_DIR = /data/kaggle/mnist
MODEL_DIR = /data/model
RESULT_DIR = .
COVERAGE = 0.95


0.95

# MNIST Train Dataset

In [8]:
val train = spark.read.format("csv")
    .option("header", true)
    .option("inferSchema", true)
    .load(PROTOCOL + DATA_DIR + "/train.csv.gz")

train = [label: int, pixel0: int ... 783 more fields]


[label: int, pixel0: int ... 783 more fields]

In [9]:
val numFeatures = train.columns.size - 1  // Remove "label" column
val numSamples = train.count

numFeatures = 784
numSamples = 42000


42000

# Dataframe for ML training

Use R formula transformation to create label and features columns for Spark ML.

In [10]:
val formula = "label ~ ."
val rformula = new RFormula()
  .setFormula(formula)
  .fit(train)

rformula.write.overwrite().save(PROTOCOL + MODEL_DIR + "/kaggle_digit_recognizer_rformula.mdl")

val trainDataDF = timed(
    "RFormula",
    rformula
        .transform(train)
        .select("label", "features")
)

println(timing)

Processing RFormula took 811 ms.



formula = label ~ .
rformula = RFormulaModel(ResolvedRFormula(label=label, terms=[pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,pixel40,pixel41,pixel42,pixel43,pixel44,pixel45,pixel46,pixel47,pixel48,pixel49,pixel50,pixel51,pixel52,pixel53,pixel54,pixel55,pixel56,pixel57,pixel58,pixel59,pixel60,pixel61,pixel62,pixel63,pixel64,pixel65,pixel66,pixel67,pixel68,pixel69,pixel70,pixel71,pixel72,pixel73,pixel74,pixel75,pixel76,pixel77,pixel78,pixel79,pixel80,pixel81,pixel82,pixel83,pi...


RFormulaModel(ResolvedRFormula(label=label, terms=[pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,pixel40,pixel41,pixel42,pixel43,pixel44,pixel45,pixel46,pixel47,pixel48,pixel49,pixel50,pixel51,pixel52,pixel53,pixel54,pixel55,pixel56,pixel57,pixel58,pixel59,pixel60,pixel61,pixel62,pixel63,pixel64,pixel65,pixel66,pixel67,pixel68,pixel69,pixel70,pixel71,pixel72,pixel73,pixel74,pixel75,pixel76,pixel77,pixel78,pixel79,pixel80,pixel81,pixel82,pixel83,pi...

In [11]:
trainDataDF.printSchema
trainDataDF.show

root
 |-- label: integer (nullable = true)
 |-- features: vector (nullable = true)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    1|(784,[132,133,134...|
|    0|(784,[122,123,124...|
|    1|(784,[124,125,126...|
|    4|(784,[146,147,148...|
|    0|(784,[121,122,123...|
|    0|(784,[124,125,126...|
|    7|(784,[202,203,204...|
|    3|(784,[177,178,179...|
|    5|(784,[153,154,155...|
|    3|(784,[119,120,121...|
|    8|(784,[180,181,182...|
|    9|(784,[182,183,184...|
|    1|(784,[154,155,156...|
|    3|(784,[144,145,146...|
|    3|(784,[122,123,124...|
|    1|(784,[156,157,158...|
|    2|(784,[148,149,150...|
|    0|(784,[129,130,131...|
|    7|(784,[206,207,208...|
|    5|(784,[121,122,123...|
+-----+--------------------+
only showing top 20 rows



# SVD

In [12]:
import org.apache.spark.sql.Row
def rowToVector(r: Row): org.apache.spark.mllib.linalg.Vector = {
    val v = r.getAs[org.apache.spark.ml.linalg.Vector](0)
    org.apache.spark.mllib.linalg.Vectors.fromML(v)
}

rowToVector: (r: org.apache.spark.sql.Row)org.apache.spark.mllib.linalg.Vector


In [13]:
val trainDataMatrix = new RowMatrix(
    trainDataDF.select("features").rdd.map(rowToVector)
)

trainDataMatrix = org.apache.spark.mllib.linalg.distributed.RowMatrix@215c7180


org.apache.spark.mllib.linalg.distributed.RowMatrix@215c7180

In [14]:
val svd: SingularValueDecomposition[RowMatrix, Matrix] = timed(
    "SVD",
    trainDataMatrix.computeSVD(numFeatures, computeU = true)
)
println(timing)

val U: RowMatrix = svd.U  // The U factor is a RowMatrix.
val s: Vector = svd.s     // The singular values are stored in a local dense vector.
val V: Matrix = svd.V     // The V factor is a local dense matrix.

Processing RFormula took 811 ms.
Processing SVD took 23324 ms.



svd = 


SingularValueDecomposition(org.apache.spark.mllib.linalg.distributed.RowMatrix@3d10b71e,[323503.703778161,109669.14717327854,101621.48745300347,93843.81264344015,87209.07876311651,78809.72055197941,72567.49168291429,64828.82470555653,64416.68658534496,58308.861751418466,55406.27915475731,54750.55160735113,49609.08790700611,49550.52462799281,47953.954194702324,46331.72758655017,44275.238616400704,43359.10907006242,41448.81945675147,40775.83066771042,39324.43779854911,38274.73752844696,37316.21577405969,36320.82031107616,35793.95790153957,34803.45941347596,34247.123321015686,33487.606203318144,32683.269877895247,31472.91710587883,30818.6...


In [15]:
val singluarValues = s.toDense.values

singluarValues = Array(323503.703778161, 109669.14717327854, 101621.48745300347, 93843.81264344015, 87209.07876311651, 78809.72055197941, 72567.49168291429, 64828.82470555653, 64416.68658534496, 58308.861751418466, 55406.27915475731, 54750.55160735113, 49609.08790700611, 49550.52462799281, 47953.954194702324, 46331.72758655017, 44275.238616400704, 43359.10907006242, 41448.81945675147, 40775.83066771042, 39324.43779854911, 38274.73752844696, 37316.21577405969, 36320.82031107616, 35793.95790153957, 34803.45941347596, 34247.123321015686, 33487.606203318144, 32683.269877895247, 31472.91710587883, 30818.644127794658, 30355.606099707322, 29404.347846111687, 29255.758990244067, 28531.531896135904, 27953.24406512744, 27100.342265174775, 26516.189399254694, 26189.552346276807, 259...


Array(323503.703778161, 109669.14717327854, 101621.48745300347, 93843.81264344015, 87209.07876311651, 78809.72055197941, 72567.49168291429, 64828.82470555653, 64416.68658534496, 58308.861751418466, 55406.27915475731, 54750.55160735113, 49609.08790700611, 49550.52462799281, 47953.954194702324, 46331.72758655017, 44275.238616400704, 43359.10907006242, 41448.81945675147, 40775.83066771042, 39324.43779854911, 38274.73752844696, 37316.21577405969, 36320.82031107616, 35793.95790153957, 34803.45941347596, 34247.123321015686, 33487.606203318144, 32683.269877895247, 31472.91710587883, 30818.644127794658, 30355.606099707322, 29404.347846111687, 29255.758990244067, 28531.531896135904, 27953.24406512744, 27100.342265174775, 26516.189399254694, 26189.552346276807, 259...

## Number of principal components (k) to cover COVERAGE

In [16]:
val windowSpec = Window
  .orderBy(desc("singluar_value"))
  .rowsBetween(Window.unboundedPreceding, Window.currentRow)
  
val windowSpecAll = Window
  .orderBy(desc("singluar_value"))
  .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

val coverageDF = sc.parallelize(singluarValues).toDF("singluar_value")
    .withColumn(
        "id",
        row_number().over(windowSpec)
    )
    .select("id", "singluar_value")
    .withColumn(
        "eignvalue", 
        pow(col("singluar_value"), 2) / lit(numSamples -1)
    )
    .withColumn(
        "cumulativeSum", 
        sum(col("eignvalue")).over(windowSpec)
    )
    .withColumn(
        "coverage", 
        col("cumulativeSum") / last(col("cumulativeSum")).over(windowSpecAll)
    )
//coverageDF.show(5)

windowSpec = org.apache.spark.sql.expressions.WindowSpec@70163cb0
windowSpecAll = org.apache.spark.sql.expressions.WindowSpec@2451d8a8
coverageDF = [id: int, singluar_value: double ... 3 more fields]


[id: int, singluar_value: double ... 3 more fields]

In [17]:
val kDF = coverageDF.where(col("coverage") >= lit(COVERAGE)).limit(1)
kDF.show

val k = kDF.select("id").collect()(0).getInt(0)

+---+------------------+------------------+-----------------+------------------+
| id|    singluar_value|         eignvalue|    cumulativeSum|          coverage|
+---+------------------+------------------+-----------------+------------------+
|102|11675.026832748754|3245.4642145147122|5443028.651091638|0.9501982263075994|
+---+------------------+------------------+-----------------+------------------+



kDF = [id: int, singluar_value: double ... 3 more fields]
k = 102


102

# PCA

In [18]:
val pca = new PCA()
  .setInputCol("features")
  .setOutputCol("pcaFeatures")
  .setK(k)
  .fit(trainDataDF)

pca = pca_63da76466e29


pca_63da76466e29

In [19]:
pca.write.overwrite().save(PROTOCOL + MODEL_DIR + "/kaggle_digit_recognizer_pca.mdl")

In [20]:
pca.explainParams()

inputCol: input column name (current: features)
k: the number of principal components (> 0) (current: 102)
outputCol: output column name (default: pca_63da76466e29__output, current: pcaFeatures)


In [21]:
pca.explainedVariance

[0.09748937689497461,0.07160266275027234,0.06145903355957282,0.05379301996327574,0.04894262134042076,0.04303213992139485,0.03277050763580546,0.028921031736972334,0.02766902346628119,0.02348871029531937,0.02099325425271383,0.020590011597844095,0.017025534993579613,0.01692787021349282,0.015811264063787012,0.01483239617649628,0.013196878946996466,0.012827270808166676,0.01187976141356012,0.011527547329596514,0.010721912242319678,0.010151993021548662,0.00964902259415799,0.009128460679541317,0.008876408591128955,0.008387663079557076,0.008118558545110089,0.007774057474595946,0.007406351163183782,0.006866614888087311,0.006579822105222276,0.0063879861144869815,0.005993670159253401,0.005889134101342426,0.005643351784057687,0.005409670481475439,0.005...

In [22]:
pca.explainedVariance.values.sum

0.9169591926906049

In [21]:
val pcaTrainDataDF = pca.transform(trainDataDF)

pcaTrainDataDF = [label: int, features: vector ... 1 more field]


[label: int, features: vector ... 1 more field]

In [22]:
pcaTrainDataDF.printSchema
//pcaTrainDataDF.show(10)

root
 |-- label: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- pcaFeatures: vector (nullable = true)



# Random Forest

In [23]:
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
val rf = new RandomForestClassifier()
  .setLabelCol("label")
  .setFeaturesCol("pcaFeatures")
  .setFeatureSubsetStrategy("auto")

rf.explainParams()

rf = rfc_ea4a7195cd70


cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. (default: false)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: auto, all,...


## Performance evaluator

In [24]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
val evaluator = new MulticlassClassificationEvaluator()
    .setLabelCol("label")
    .setPredictionCol("prediction")
    .setMetricName("accuracy")

evaluator = mcEval_af99de95a8a5


mcEval_af99de95a8a5

## Grid search parameters

In [25]:
import org.apache.spark.ml.tuning.ParamGridBuilder
val params = new ParamGridBuilder()
    .addGrid(rf.maxDepth, Array(10, 15, 20))
    .addGrid(rf.numTrees, Array(100, 150))
    .addGrid(rf.subsamplingRate, Array(0.3, 0.5, 0.7))
  .build()

params = 


Array({
	rfc_ea4a7195cd70-maxDepth: 5,
	rfc_ea4a7195cd70-numTrees: 50,
	rfc_ea4a7195cd70-subsamplingRate: 0.1
}, {
	rfc_ea4a7195cd70-maxDepth: 5,
	rfc_ea4a7195cd70-numTrees: 100,
	rfc_ea4a7195cd70-subsamplingRate: 0.1
}, {
	rfc_ea4a7195cd70-maxDepth: 10,
	rfc_ea4a7195cd70-numTrees: 50,
	rfc_ea4a7195cd70-subsamplingRate: 0.1
}, {
	rfc_ea4a7195cd70-maxDepth: 10,
	rfc_ea4a7195cd70-numTrees: 100,
	rfc_ea4a7195cd70-subsamplingRate: 0.1
}, {
	rfc_ea4a7195cd70-maxDepth: 15,
	rfc_ea4a7195cd70-numTrees: 50,
	rfc_ea4a7195cd70-subsamplingRate: 0.1
}, {
	rfc_ea4a7195cd70-maxDepth: 15,
	rfc_ea4a7195cd70-numTrees: 100,
	rfc_ea4a7195cd70-subsamplingRate: 0.1
}, {
	rfc_ea4a7195cd70-maxDepth: 5,
	rfc_e...


## [Cross Validation](https://spark.apache.org/docs/latest/ml-tuning.html#cross-validation)

In [ ]:
import org.apache.spark.ml.tuning.{CrossValidator, CrossValidatorModel}
val cv = new TrainValidationSplit()
  .setEstimatorParamMaps(params)
  .setEstimator(rf)
  .setEvaluator(evaluator)
  .setNumFolds(CROSS_VALIDATE_NUM_HOLDS)  // Use 3+ in practice
  .setParallelism(CROSS_VALIDATE_PARALLELISM)  // Evaluate up to 2 parameter settings in parallel

val cvFitted = timed(
    "Cross validation grid search",
    cv.fit(pcaTrainDataDF)
)

cvFitted.write.overwrite().save(PROTOCOL + MODEL_DIR + "/kaggle_digit_recognizer_cv.mdl")

In [ ]:
val fitted = cvFitted

## [Train Validate Split](https://spark.apache.org/docs/latest/ml-tuning.html#train-validation-split)

In [27]:
import org.apache.spark.ml.tuning.{TrainValidationSplit, TrainValidationSplitModel}
val tvs = new TrainValidationSplit()
  .setTrainRatio(0.75) // also the default.
  .setEstimatorParamMaps(params)
  .setEstimator(rf)
  .setEvaluator(evaluator)

val tvsFitted = timed(
    "Train validation split grid search",
    tvs.fit(pcaTrainDataDF)
)

tvsFitted.write.overwrite().save(PROTOCOL + MODEL_DIR + "/kaggle_digit_recognizer_tvs.mdl")

tvsFitted = tvs_c6aadc18f7f1


tvs_c6aadc18f7f1

In [ ]:
val fitted = tvsFitted

# Model performance

In [28]:
val model = fitted.bestModel.asInstanceOf[RandomForestClassificationModel]
model.explainParams()
println(timing)
println(s"max depth ${model.getMaxDepth}")
println(s"num trees ${model.getNumTrees}")
println(s"impurity  ${model.getImpurity}")
println(s"subsamplingRate ${model.getSubsamplingRate}")

Processing RFormula took 740 ms.
Processing SVD took 25087 ms.
Processing GridSearch took 931541 ms.

max depth 15
num trees 100
impurity  gini
subsamplingRate rfc_ea4a7195cd70__subsamplingRate


trainedModel = RandomForestClassificationModel (uid=rfc_ea4a7195cd70) with 100 trees


RandomForestClassificationModel (uid=rfc_ea4a7195cd70) with 100 trees

In [29]:
model.write.overwrite().save(PROTOCOL + MODEL_DIR + "/kaggle_digit_recognizer_rf.mdl")

In [30]:
val predictions = fitted.transform(pcaTrainDataDF)

predictions = [label: int, features: vector ... 4 more fields]


[label: int, features: vector ... 4 more fields]

In [31]:
predictions.show(10)

+-----+--------------------+--------------------+--------------------+--------------------+----------+
|label|            features|         pcaFeatures|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+----------+
|    1|(784,[132,133,134...|[103.738813757989...|[0.00450894878577...|[4.50894878577726...|       1.0|
|    0|(784,[122,123,124...|[2466.78627830941...|[84.5164983198871...|[0.84516498319887...|       0.0|
|    1|(784,[124,125,126...|[-121.55984060477...|[0.03951360973791...|[3.95136097379186...|       1.0|
|    4|(784,[146,147,148...|[599.578991071955...|[1.39702870431335...|[0.01397028704313...|       4.0|
|    0|(784,[121,122,123...|[2689.04430947599...|[91.5365703890843...|[0.91536570389084...|       0.0|
|    0|(784,[124,125,126...|[1253.08650413365...|[50.3024157705876...|[0.50302415770587...|       0.0|
|    7|(784,[202,203,204...|[93.0114290617944...|[0.03352711302764...|[3.

In [32]:
val accuracy = fitted.getEvaluator.evaluate(predictions)
println(s"Test Error = ${(1.0 - accuracy)}")

Test Error = 0.005428571428571449


accuracy = 0.9945714285714286


0.9945714285714286

# Test

In [63]:
val test = spark.read.format("csv")
    .option("header", true)
    .option("inferSchema", true)
    .load(PROTOCOL + DATA_DIR + "/test.csv.gz")

test = [pixel0: int, pixel1: int ... 782 more fields]


[pixel0: int, pixel1: int ... 782 more fields]

In [65]:
val testDataDF = rformula
    .transform(test)
    .select("features")

testDataDF = [features: vector]


[features: vector]

In [72]:
testDataDF
    .show()

+--------------------+
|            features|
+--------------------+
|(784,[122,123,124...|
|(784,[179,180,181...|
|(784,[181,182,183...|
|(784,[208,210,211...|
|(784,[95,96,97,98...|
|(784,[202,203,204...|
|(784,[124,125,126...|
|(784,[122,123,124...|
|(784,[153,154,155...|
|(784,[121,122,123...|
|(784,[155,156,157...|
|(784,[179,180,181...|
|(784,[147,148,149...|
|(784,[126,127,128...|
|(784,[177,178,204...|
|(784,[120,121,122...|
|(784,[119,120,121...|
|(784,[125,126,153...|
|(784,[179,180,181...|
|(784,[123,124,125...|
+--------------------+
only showing top 20 rows



In [67]:
val pcaTestDataDF = pca.transform(testDataDF)

pcaTestDataDF = [features: vector, pcaFeatures: vector]


[features: vector, pcaFeatures: vector]

In [68]:
pcaTestDataDF.printSchema

root
 |-- features: vector (nullable = true)
 |-- pcaFeatures: vector (nullable = true)



In [69]:
pcaTestDataDF.show(3)

+--------------------+--------------------+
|            features|         pcaFeatures|
+--------------------+--------------------+
|(784,[122,123,124...|[1612.42042408780...|
|(784,[179,180,181...|[1919.58554625988...|
|(784,[181,182,183...|[258.514350043858...|
+--------------------+--------------------+
only showing top 3 rows



In [70]:
val validations = fitted.transform(pcaTestDataDF)

validations = [features: vector, pcaFeatures: vector ... 3 more fields]


[features: vector, pcaFeatures: vector ... 3 more fields]

In [71]:
validations.show()

+--------------------+--------------------+--------------------+--------------------+----------+
|            features|         pcaFeatures|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+--------------------+----------+
|(784,[122,123,124...|[1612.42042408780...|[3.10440378782087...|[0.03104403787820...|       2.0|
|(784,[179,180,181...|[1919.58554625988...|[89.8726237980590...|[0.89872623798059...|       0.0|
|(784,[181,182,183...|[258.514350043858...|[3.36094066825166...|[0.03360940668251...|       9.0|
|(784,[208,210,211...|[497.736897425884...|[7.46759295209640...|[0.07467592952096...|       4.0|
|(784,[95,96,97,98...|[667.552229232708...|[2.55710185908117...|[0.02557101859081...|       3.0|
|(784,[202,203,204...|[487.203794732073...|[2.54427274091337...|[0.02544272740913...|       7.0|
|(784,[124,125,126...|[2481.24718102618...|[82.1917772496175...|[0.82191777249617...|       0.0|
|(784,[122,123,124...|[1010.70